#Bibliotecas

Este código utiliza bibliotecas/ferramentas, como nltk e demoji para a limpeza e normalização dos dados:

**Documentações:**

- **nltk:** https://www.nltk.org/howto/portuguese_en.html
- **demoji**: https://pypi.org/project/demoji/
- **Enelvo**: https://github.com/thalesbertaglia/enelvo
- **re:** https://docs.python.org/3/library/re.html

Para criptografia do arquivo com usuários:
- **cryptography:** https://pypi.org/project/cryptography/

In [ ]:
import nltk
from nltk.tokenize import regexp_tokenize


nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')
stopwords.append("pra")
stopwords.append("tá")

import re

!pip install demoji
import demoji

!pip install enelvo
from enelvo import normaliser

from cryptography.fernet import Fernet
import os

#Pipeline

Para criptografar o arquivo de usuários usamos a biblioteca cryptography e criamos algumas funções:

In [ ]:
# Gera uma chave usando o módulo Fernet e salva em um arquivo chamado "mykey.key"
def generate_key():
    key = Fernet.generate_key()
    with open("mykey.key", "wb") as key_file:
        key_file.write(key)

# Carrega a chave gerada anteriormente do arquivo "mykey.key"
def load_key():
    return open("mykey.key", "rb").read()

# Encripta o conteúdo do arquivo especificado usando a chave fornecida
def encrypt(filename, key):
    f = Fernet(key)
    with open(filename, "rb") as file:
        file_data = file.read()
        encrypted_data = f.encrypt(file_data)
    with open(filename, "wb") as file:
        file.write(encrypted_data)

# Decripta o conteúdo do arquivo especificado usando a chave fornecida
def decrypt(filename, key):
    f = Fernet(key)
    with open(filename, "rb") as file:
        encrypted_data = file.read()
        try:
            decrypted_data = f.decrypt(encrypted_data)
        except:
            print("Invalid key")
            return
    with open(filename, "wb") as file:
        file.write(decrypted_data)

Os dados foram anonimizados para proteger a privacidade dos usuários substituindo usuários (@...) por chaves no arquivo de dicionário criptografado (user_dict).

Além disso, tokenizamos as palavras (sem separar datas, hashtags e números com vírgula), realizamos a conversão de caracteres maiúsculos para minúsculos, substituímos emojis (Descrição ou emojipositivo/emojinegativo), removemos URLs, pontuações e palavras irrelevantes (stop words) e

Utilizamos a ferramenta enelvo para corrigir erros ortográficos.


In [71]:
emoji_list = {':))': 'emojipositivo', ':)': 'emojipositivo', ':d': 'emojipositivo', ':p': 'emojipositivo',
              ':(': 'emojinegativo', ':((': 'emojinegativo', '8)': 'emojineutro'}
normalizador = normaliser.Normaliser(tokenizer='readable')

# Função para substituir usuários em um texto usando um dicionário salvo em 'user_dict.txt'
def substitui_user(match):
  loaded_key= load_key()
  user = match.group(0)

  if os.stat('user_dict.txt').st_size == 0:
    with open('user_dict.txt', 'r+') as dic:
      lines = dic.readlines()
      nuser = f'{user}:user{len(lines) + 1}\n'
      dic.write(nuser)
      dic.close()

      encrypt('user_dict.txt', loaded_key)
      return f'user{len(lines) + 1}'

  else:
    decrypt('user_dict.txt', loaded_key)
    with open('user_dict.txt', 'r+') as dic:
      lines = dic.readlines()
      for line in lines:
        key, value = line.strip().split(":")
        if key == user:
          dic.close()
          encrypt('user_dict.txt', loaded_key)
          return value

      nuser = f'{user}:user{len(lines) + 1}\n'
      dic.write(nuser)

      dic.close()
      encrypt('user_dict.txt', loaded_key)
      return f'user{len(lines) + 1}'

# Função para substituir emojis
def substitui_emoji(text):
    for emoji, label in emoji_list.items():
        text = text.replace(emoji, label)
    dem = demoji.findall(text)
    for item, value in dem.items():
        text = text.replace(item, f" {value.replace(' ', '')}")
    return text

def preprocess(texto, remove_stopwords=False, tokenization=False):
    # Substituindo , por chavev
    texto = texto.str.replace(r',', 'chavevirg')

    # Normalizando texto
    texto = texto.apply(lambda x: normalizador.normalise(x))

    # Substituindo emojis
    texto = texto.apply(substitui_emoji)

    # Substituindo users
    texto = texto.str.replace(r'@\w+', substitui_user)

    # Removendo URLs
    texto = texto.str.replace(r'https?://\S+', '')

    # Substituindo chavev por ,
    texto = texto.str.replace(r'chavevirg', ',')

    # Removendo vírgulas não associadas a números e outras pontuações
    texto = texto.apply(lambda x: re.sub(r'(?<!\d),(?=\D)|(?<=\D),(?!\d)|(?<!\d),(?=\d)|(?<!\d)\/|\/(?!\d)|_|[^\w#\/\s,]',
                                         '', x))

    # Removendo Stop Words (opcional)
    if remove_stopwords:
        texto = texto.apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords]))

    # Tokenization (opcional)
    if tokenization:
        pattern = r"\b(?:\d{1,2}/\d{1,2}(?:/\d{4})?)\b|(?:\d+,\d+)|\b\w+\b|#\w+\b"
        texto = texto.apply(lambda x: regexp_tokenize(x, pattern))

    return texto